In [15]:
!pip -q install duckdb

from google.colab import files
print("Upload: walmart_sales_clean.csv and data_dictionary.csv")
uploaded = files.upload()

import os, duckdb

def pick(*names):
    for n in names:
        if n in uploaded or os.path.exists(n):
            return n
    return None

clean_path = pick('walmart_sales_clean.csv', 'walmart_sales_clean (1).csv')
dict_path  = pick('data_dictionary.csv', 'data_dictionary (1).csv')

assert clean_path, "Upload walmart_sales_clean.csv"
print("Detected:", {"clean": clean_path, "dictionary": dict_path})

con = duckdb.connect(database='walmart.duckdb', read_only=False)

# Creating a physical table from the cleaned CSV
con.execute(f"""
    CREATE OR REPLACE TABLE sales_clean AS
    SELECT * FROM read_csv_auto('{clean_path}', dateformat='%Y-%m-%d', header=True);
""")

# Peeking at columns
print(con.execute("PRAGMA show_tables").fetchdf())
print(con.execute("DESCRIBE sales_clean").fetchdf())


Upload: walmart_sales_clean.csv and data_dictionary.csv


Saving data_dictionary.csv to data_dictionary (1).csv
Saving walmart_sales_clean.csv to walmart_sales_clean (1).csv
Detected: {'clean': 'walmart_sales_clean.csv', 'dictionary': 'data_dictionary.csv'}


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

          name
0  sales_clean
        column_name column_type null   key default extra
0             Store      BIGINT  YES  None    None  None
1              Dept      BIGINT  YES  None    None  None
2              Date        DATE  YES  None    None  None
3      Weekly_Sales      DOUBLE  YES  None    None  None
4         IsHoliday     BOOLEAN  YES  None    None  None
5            Year_x      BIGINT  YES  None    None  None
6           Month_x      BIGINT  YES  None    None  None
7      Month_Name_x     VARCHAR  YES  None    None  None
8            Week_x      BIGINT  YES  None    None  None
9         Quarter_x      BIGINT  YES  None    None  None
10      Temperature      DOUBLE  YES  None    None  None
11       Fuel_Price      DOUBLE  YES  None    None  None
12        MarkDown1      DOUBLE  YES  None    None  None
13        MarkDown2      DOUBLE  YES  None    None  None
14        MarkDown3      DOUBLE  YES  None    None  None
15        MarkDown4      DOUBLE  YES  None    None  None
1

Analysis Queries

In [16]:
!pip -q install duckdb
import duckdb, os

DB_PATH = "/content/walmart.duckdb"
con = duckdb.connect(database=DB_PATH, read_only=False)

print("Connected to:", DB_PATH)


Connected to: /content/walmart.duckdb


In [17]:
CSV_PATH = "/content/walmart_sales_clean.csv"

con.execute(f"""
    CREATE OR REPLACE TABLE sales_clean AS
    SELECT * FROM read_csv_auto('{CSV_PATH}', dateformat='%Y-%m-%d', header=True);
""")
print(con.execute("PRAGMA show_tables").df())
print(con.execute("DESCRIBE sales_clean").df().head(30))


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

          name
0  sales_clean
        column_name column_type null   key default extra
0             Store      BIGINT  YES  None    None  None
1              Dept      BIGINT  YES  None    None  None
2              Date        DATE  YES  None    None  None
3      Weekly_Sales      DOUBLE  YES  None    None  None
4         IsHoliday     BOOLEAN  YES  None    None  None
5            Year_x      BIGINT  YES  None    None  None
6           Month_x      BIGINT  YES  None    None  None
7      Month_Name_x     VARCHAR  YES  None    None  None
8            Week_x      BIGINT  YES  None    None  None
9         Quarter_x      BIGINT  YES  None    None  None
10      Temperature      DOUBLE  YES  None    None  None
11       Fuel_Price      DOUBLE  YES  None    None  None
12        MarkDown1      DOUBLE  YES  None    None  None
13        MarkDown2      DOUBLE  YES  None    None  None
14        MarkDown3      DOUBLE  YES  None    None  None
15        MarkDown4      DOUBLE  YES  None    None  None
1

In [19]:
con.execute("""
CREATE OR REPLACE VIEW sales_master AS
SELECT
  Store,
  Dept,
  Date,
  Weekly_Sales,
  IsHoliday,
  Temperature,
  Fuel_Price,
  CPI,
  Unemployment,
  Type,
  Size,
  SalesperSqft,
  Total_Markdowns,
  Holiday_Period
FROM sales_clean
""")
print(" sales_master view created")


✅ sales_master view created


1.Holiday Impact

In [20]:
# Overall holiday vs non-holiday
con.execute("""
SELECT IsHoliday,
       AVG(Weekly_Sales) AS avg_sales,
       SUM(Weekly_Sales) AS total_sales
FROM sales_master
GROUP BY IsHoliday
ORDER BY IsHoliday DESC
""").df()


,IsHoliday,avg_sales,total_sales
0,True,17035.823187,5.052996e+08
1,False,15901.445069,6.231919e+09


In [21]:
# Holiday lift %
con.execute("""
WITH agg AS (
  SELECT AVG(CASE WHEN IsHoliday THEN Weekly_Sales END)     AS avg_holiday,
         AVG(CASE WHEN NOT IsHoliday THEN Weekly_Sales END) AS avg_nonholiday
  FROM sales_master
)
SELECT avg_holiday, avg_nonholiday,
       (avg_holiday - avg_nonholiday) AS diff,
       100.0*(avg_holiday - avg_nonholiday)/avg_nonholiday AS pct_lift
FROM agg
""").df()


,avg_holiday,avg_nonholiday,diff,pct_lift
0,17035.823187,15901.445069,1134.378118,7.133805


In [22]:
# Breakdown by specific holiday periods (if Holiday_Period is populated)
con.execute("""
SELECT Holiday_Period,
       AVG(Weekly_Sales) AS avg_sales,
       SUM(Weekly_Sales) AS total_sales,
       COUNT(*)          AS rows
FROM sales_master
WHERE Holiday_Period IS NOT NULL AND Holiday_Period <> ''
GROUP BY Holiday_Period
ORDER BY total_sales DESC
""").df()


,Holiday_Period,avg_sales,total_sales,rows
0,Regular,15901.445069,6.231919e+09,391909
1,Holiday,17035.823187,5.052996e+08,29661


2. Store efficiency

In [23]:
con.execute("""
WITH base AS (
  SELECT Store, Type, Size,
         SUM(Weekly_Sales) AS total_sales,
         AVG(Weekly_Sales) AS avg_weekly_sales,
         SUM(Weekly_Sales)/NULLIF(Size, 0) AS sales_per_sqft
  FROM sales_master
  GROUP BY Store, Type, Size
)
SELECT *,
       ROW_NUMBER() OVER (ORDER BY total_sales DESC)    AS rank_total_sales,
       ROW_NUMBER() OVER (ORDER BY sales_per_sqft DESC) AS rank_sales_per_sqft,
       ROW_NUMBER() OVER (ORDER BY avg_weekly_sales DESC) AS rank_avg_weekly
FROM base
ORDER BY rank_total_sales
""").df()


,Store,Type,Size,total_sales,avg_weekly_sales,sales_per_sqft,rank_total_sales,rank_sales_per_sqft,rank_avg_weekly
0,20,A,203742,3.013978e+08,29508.301592,1479.311053,1,7,1
1,4,A,205863,2.995440e+08,29161.210415,1455.064550,2,10,2
2,14,A,200898,2.889999e+08,28784.851727,1438.540510,3,11,3
3,13,A,219622,2.865177e+08,27355.136891,1304.594730,4,16,4
4,2,A,202307,2.753824e+08,26898.070031,1361.210640,5,14,5
5,10,B,126512,2.716177e+08,26332.303819,2146.971939,6,2,6
6,27,A,204184,2.538559e+08,24826.984536,1243.270368,7,22,7
7,6,A,202505,2.237561e+08,21913.243624,1104.941264,8,26,8
8,1,A,151315,2.224028e+08,21710.543621,1469.800144,9,8,9
9,39,A,184109,2.074455e+08,21000.763562,1126.753947,10,25,10


3. Department Mix

In [24]:
# Overall contribution
con.execute("""
SELECT Dept,
       SUM(Weekly_Sales) AS dept_sales,
       100.0*SUM(Weekly_Sales)/SUM(SUM(Weekly_Sales)) OVER () AS pct_contribution
FROM sales_master
GROUP BY Dept
ORDER BY dept_sales DESC
""").df()


,Dept,dept_sales,pct_contribution
0,92,4.839433e+08,7.183132e+00
1,95,4.493202e+08,6.669223e+00
2,38,3.931181e+08,5.835021e+00
3,72,3.057252e+08,4.537854e+00
4,90,2.910685e+08,4.320306e+00
...,...,...,...
76,51,3.057283e+04,4.537901e-04
77,78,1.714710e+03,2.545130e-05
78,39,1.779800e+02,2.641743e-06
79,43,1.432000e+01,2.125506e-07


In [25]:
# By store
con.execute("""
SELECT Store, Dept,
       SUM(Weekly_Sales) AS dept_sales,
       100.0*SUM(Weekly_Sales)
         / SUM(SUM(Weekly_Sales)) OVER (PARTITION BY Store) AS pct_in_store
FROM sales_master
GROUP BY Store, Dept
ORDER BY Store, dept_sales DESC
""").df()


,Store,Dept,dept_sales,pct_in_store
0,1,92,19370632.64,8.709707
1,1,95,17270404.89,7.765372
2,1,90,11787139.34,5.299906
3,1,38,11436885.83,5.142420
4,1,93,10252983.08,4.610096
...,...,...,...,...
3326,45,45,377.34,0.000336
3327,45,51,104.52,0.000093
3328,45,78,88.00,0.000078
3329,45,96,5.94,0.000005


4. Economic correlations

In [26]:
# Overall correlations
con.execute("""
SELECT
  corr(Weekly_Sales, Fuel_Price)   AS corr_sales_fuel,
  corr(Weekly_Sales, CPI)          AS corr_sales_cpi,
  corr(Weekly_Sales, Unemployment) AS corr_sales_unemp,
  corr(Weekly_Sales, Temperature)  AS corr_sales_temp
FROM sales_master
""").df()


,corr_sales_fuel,corr_sales_cpi,corr_sales_unemp,corr_sales_temp
0,-0.00012,-0.020921,-0.025864,-0.002312


In [27]:
# By store type
con.execute("""
SELECT Type,
       corr(Weekly_Sales, Fuel_Price)   AS corr_sales_fuel,
       corr(Weekly_Sales, CPI)          AS corr_sales_cpi,
       corr(Weekly_Sales, Unemployment) AS corr_sales_unemp,
       corr(Weekly_Sales, Temperature)  AS corr_sales_temp
FROM sales_master
GROUP BY Type
ORDER BY Type
""").df()


,Type,corr_sales_fuel,corr_sales_cpi,corr_sales_unemp,corr_sales_temp
0,A,-0.008223,0.022002,-0.025493,-0.013339
1,B,0.041898,-0.193594,0.033820,0.013341
2,C,-0.011570,0.072300,0.005286,0.033590


5. KPI

In [28]:
# Monthly totals
con.execute("""
SELECT date_trunc('month', Date) AS month,
       SUM(Weekly_Sales)         AS monthly_sales
FROM sales_master
GROUP BY month
ORDER BY month
""").df()


,month,monthly_sales
0,2010-02-01,1.903330e+08
1,2010-03-01,1.819198e+08
2,2010-04-01,2.314124e+08
3,2010-05-01,1.867109e+08
4,2010-06-01,1.922462e+08
5,2010-07-01,2.325801e+08
6,2010-08-01,1.876401e+08
7,2010-09-01,1.772679e+08
8,2010-10-01,2.171618e+08
9,2010-11-01,2.028534e+08


In [29]:
# Quarterly totals
con.execute("""
SELECT date_trunc('quarter', Date) AS quarter,
       SUM(Weekly_Sales)           AS quarterly_sales
FROM sales_master
GROUP BY quarter
ORDER BY quarter
""").df()


,quarter,quarterly_sales
0,2010-01-01,3.722528e+08
1,2010-04-01,6.103695e+08
2,2010-07-01,5.974881e+08
3,2010-10-01,7.087757e+08
4,2011-01-01,5.293917e+08
5,2011-04-01,5.979481e+08
6,2011-07-01,6.393585e+08
7,2011-10-01,6.815017e+08
8,2012-01-01,5.924677e+08
9,2012-04-01,6.182977e+08


In [30]:
# YTD cumulative
con.execute("""
WITH daily AS (
  SELECT Date,
         EXTRACT(YEAR FROM Date) AS yr,
         SUM(Weekly_Sales) AS sales_day
  FROM sales_master
  GROUP BY Date
)
SELECT Date, yr,
       SUM(sales_day) OVER (PARTITION BY yr ORDER BY Date
                            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS ytd_sales
FROM daily
ORDER BY Date
""").df()


,Date,yr,ytd_sales
0,2010-02-05,2010,4.975074e+07
1,2010-02-12,2010,9.808742e+07
2,2010-02-19,2010,1.463644e+08
3,2010-02-26,2010,1.903330e+08
4,2010-03-05,2010,2.372045e+08
...,...,...,...
138,2012-09-28,2012,1.815771e+09
139,2012-10-05,2012,1.863338e+09
140,2012-10-12,2012,1.909466e+09
141,2012-10-19,2012,1.954589e+09


In [32]:
!pip -q install openpyxl


In [33]:
import pandas as pd

# Holiday impact
q_holiday_overall = con.execute("""
SELECT IsHoliday,
       AVG(Weekly_Sales) AS avg_sales,
       SUM(Weekly_Sales) AS total_sales
FROM sales_master
GROUP BY IsHoliday
ORDER BY IsHoliday DESC
""").df()

q_holiday_lift = con.execute("""
WITH agg AS (
  SELECT AVG(CASE WHEN IsHoliday THEN Weekly_Sales END)     AS avg_holiday,
         AVG(CASE WHEN NOT IsHoliday THEN Weekly_Sales END) AS avg_nonholiday
  FROM sales_master
)
SELECT avg_holiday, avg_nonholiday,
       (avg_holiday - avg_nonholiday) AS diff,
       100.0*(avg_holiday - avg_nonholiday)/avg_nonholiday AS pct_lift
FROM agg
""").df()

q_holiday_by_period = con.execute("""
SELECT Holiday_Period,
       AVG(Weekly_Sales) AS avg_sales,
       SUM(Weekly_Sales) AS total_sales,
       COUNT(*)          AS rows
FROM sales_master
WHERE Holiday_Period IS NOT NULL AND Holiday_Period <> ''
GROUP BY Holiday_Period
ORDER BY total_sales DESC
""").df()

# Store efficiency
q_store_eff = con.execute("""
WITH base AS (
  SELECT Store, Type, Size,
         SUM(Weekly_Sales) AS total_sales,
         AVG(Weekly_Sales) AS avg_weekly_sales,
         SUM(Weekly_Sales)/NULLIF(Size, 0) AS sales_per_sqft
  FROM sales_master
  GROUP BY Store, Type, Size
)
SELECT *,
       ROW_NUMBER() OVER (ORDER BY total_sales DESC)    AS rank_total_sales,
       ROW_NUMBER() OVER (ORDER BY sales_per_sqft DESC) AS rank_sales_per_sqft,
       ROW_NUMBER() OVER (ORDER BY avg_weekly_sales DESC) AS rank_avg_weekly
FROM base
ORDER BY rank_total_sales
""").df()

# Department mix
q_dept_overall = con.execute("""
SELECT Dept,
       SUM(Weekly_Sales) AS dept_sales,
       100.0*SUM(Weekly_Sales)/SUM(SUM(Weekly_Sales)) OVER () AS pct_contribution
FROM sales_master
GROUP BY Dept
ORDER BY dept_sales DESC
""").df()

q_dept_by_store = con.execute("""
SELECT Store, Dept,
       SUM(Weekly_Sales) AS dept_sales,
       100.0*SUM(Weekly_Sales)
         / SUM(SUM(Weekly_Sales)) OVER (PARTITION BY Store) AS pct_in_store
FROM sales_master
GROUP BY Store, Dept
ORDER BY Store, dept_sales DESC
""").df()

# Economic correlations
q_corr_overall = con.execute("""
SELECT
  corr(Weekly_Sales, Fuel_Price)   AS corr_sales_fuel,
  corr(Weekly_Sales, CPI)          AS corr_sales_cpi,
  corr(Weekly_Sales, Unemployment) AS corr_sales_unemp,
  corr(Weekly_Sales, Temperature)  AS corr_sales_temp
FROM sales_master
""").df()

q_corr_by_type = con.execute("""
SELECT Type,
       corr(Weekly_Sales, Fuel_Price)   AS corr_sales_fuel,
       corr(Weekly_Sales, CPI)          AS corr_sales_cpi,
       corr(Weekly_Sales, Unemployment) AS corr_sales_unemp,
       corr(Weekly_Sales, Temperature)  AS corr_sales_temp
FROM sales_master
GROUP BY Type
ORDER BY Type
""").df()

# KPI rollups
q_monthly = con.execute("""
SELECT date_trunc('month', Date) AS month,
       SUM(Weekly_Sales)         AS monthly_sales
FROM sales_master
GROUP BY month
ORDER BY month
""").df()

q_quarterly = con.execute("""
SELECT date_trunc('quarter', Date) AS quarter,
       SUM(Weekly_Sales)           AS quarterly_sales
FROM sales_master
GROUP BY quarter
ORDER BY quarter
""").df()

q_ytd = con.execute("""
WITH daily AS (
  SELECT Date,
         EXTRACT(YEAR FROM Date) AS yr,
         SUM(Weekly_Sales) AS sales_day
  FROM sales_master
  GROUP BY Date
)
SELECT Date, yr,
       SUM(sales_day) OVER (PARTITION BY yr ORDER BY Date
                            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS ytd_sales
FROM daily
ORDER BY Date
""").df()


In [34]:
output_file = "walmart_analysis_outputs.xlsx"

with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    q_holiday_overall.to_excel(writer, sheet_name="Holiday_Overall", index=False)
    q_holiday_lift.to_excel(writer, sheet_name="Holiday_Lift", index=False)
    q_holiday_by_period.to_excel(writer, sheet_name="Holiday_By_Period", index=False)
    q_store_eff.to_excel(writer, sheet_name="Store_Efficiency", index=False)
    q_dept_overall.to_excel(writer, sheet_name="Dept_Overall", index=False)
    q_dept_by_store.to_excel(writer, sheet_name="Dept_By_Store", index=False)
    q_corr_overall.to_excel(writer, sheet_name="Corr_Overall", index=False)
    q_corr_by_type.to_excel(writer, sheet_name="Corr_By_Type", index=False)
    q_monthly.to_excel(writer, sheet_name="Monthly_Sales", index=False)
    q_quarterly.to_excel(writer, sheet_name="Quarterly_Sales", index=False)
    q_ytd.to_excel(writer, sheet_name="YTD_Sales", index=False)

print("Excel file created:", output_file)


Excel file created: walmart_analysis_outputs.xlsx


In [35]:
from google.colab import files
files.download("walmart_analysis_outputs.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>